<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-PyTorch/blob/main/Comet_and_Pytorch_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://cdn.comet.ml/img/notebook_logo.png">

[PyTorch Lightning](https://pytorch-lightning.readthedocs.io/en/latest/) helps organize PyTorch code and decouple the science code from the engineering code. It’s more of a style-guide than a framework. By organizing PyTorch code under a LightningModule, Lightning makes things like TPU, multi-GPU and 16-bit precision training (40+ other features) trivial.

The Pytorch Lightning `CometLogger` allows you to easily track your model hyperparameters, metrics, assets and much more! You can find more information about this integration [here](https://www.comet.ml/docs/v2/integrations/ml-frameworks/pytorch-lightning/)

Curious about how Comet can help you build better models, faster? Find out more about [Comet](https://www.comet.com/site/products/ml-experiment-tracking/?utm_campaign=pytorch-lightning&utm_medium=colab) and our [other integrations](https://www.comet.ml/docs/v2/integrations/overview/)

Get a preview for what's to come. Check out a completed experiment created from this notebook [here](https://www.comet.com/examples/comet-examples-pytorch-lightning/view/NDKqB4g6Beqi6DaXkCu2DQJzz/panels).


# Install Comet

In [ ]:
%pip install comet_ml

# Install Dependencies

In [ ]:
%pip install torch torchvision "pytorch-lightning<2.0.0"

# Initialize Comet

In [3]:
import comet_ml
comet_ml.init(project_name="comet-examples-pytorch-lightning")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


# Create the `CometLogger`

In [4]:
import os
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CometLogger

# arguments made to CometLogger are passed on to the comet_ml.Experiment class
comet_logger = CometLogger()

INFO:pytorch_lightning.loggers.comet:CometLogger will be initialized in online mode


# Define Lightning Template

In [5]:
import torch
import torch.nn.functional as F
import pytorch_lightning as pl

from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader


class Model(pl.LightningModule):
    def __init__(self, layer_size=784):
        super().__init__()
        self.save_hyperparameters()
        self.l1 = torch.nn.Linear(layer_size, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_nb):
        x, y = batch
        y_hat = self.forward(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

# Define Training Parameters

In [6]:
import os

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64

# Run Training

In [7]:
# Init our model
model = Model()

# Init DataLoader from MNIST Dataset
train_ds = MNIST(
    PATH_DATASETS, train=True, download=True, transform=transforms.ToTensor()
)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

eval_ds = MNIST(
    PATH_DATASETS, train=False, download=True, transform=transforms.ToTensor()
)
eval_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

comet_logger.log_hyperparams({"batch_size": BATCH_SIZE})

# Initialize a trainer
trainer = Trainer(gpus=AVAIL_GPUS, max_epochs=3, logger=comet_logger)

# Train the model ⚡
trainer.fit(model, train_loader, eval_loader)

100%|██████████| 9912422/9912422 [00:00<00:00, 418898853.85it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 43139492.10it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 175780078.20it/s]


...

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/tirendaz/comet-examples-pytorch-lightning/b37fe88ae744422498cfd24f11c16d88
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [71]       : (0.4166260361671448, 2.290541887283325)
COMET INFO:     train_loss [14] : (0.5117462277412415, 0.994901180267334)
COMET INFO:     val_loss [3]    : (0.6652342081069946, 0.6984639763832092)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 256
COMET INFO:     layer_size : 784
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO: